# Determine effect of river Lek near Schoonhoven

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import flopy
import nlmod
logger = nlmod.util.get_color_logger("INFO");

In [ ]:
model_name = "Schoonhoven"
model_ws = "schoonhoven"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
extent = [116_500, 120_000, 439_000, 442_000]
time = pd.date_range("2020", "2023", freq="MS")  # monthly timestep

## Build model

In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent,
    use_regis=True,
    use_geotop=False,
    cachedir=cachedir,
    cachename="layer_model.nc",
)
layer_model

In [ ]:
ds = nlmod.to_model_ds(layer_model, model_name, model_ws, delr=50.0)
ds = nlmod.time.set_ds_time(ds, time=time)

In [ ]:
f, ax = plt.subplots(figsize=(13, 5))
line = [(116_500, 440_000), (120_000, 440_000)]
dcs = nlmod.plot.DatasetCrossSection(ds, line, ax=ax, zmin=-100)
colors = nlmod.read.regis.get_legend()['color'].to_dict()
for layer in ds.layer.data:
    if layer not in colors:
        colors[layer] = colors['HLc']
dcs.plot_layers(colors=colors, min_label_area=1000.0);
dcs.plot_grid(vertical=False)

Exercise 1: plot the horizontal conductivity (kh) in a cross-section along the same line. Use the method `dcs.plot_array()` to plot the variable 'kh' in ds.

## Get surface water data
Get information about the surface water data. Run the notebook surface_water.ipynb for more information.

In [ ]:
fname = os.path.join(cachedir, 'bgt.geojson')
if not os.path.isfile(fname):
    bgt = nlmod.read.bgt.get_bgt(extent)
    la = nlmod.gwf.surface_water.download_level_areas(bgt, extent=extent, raise_exceptions=False)
    bgt = nlmod.gwf.surface_water.add_stages_from_waterboards(bgt, la=la)
    bgt.to_file(fname, driver="GeoJSON")
else:
    logger.info('using cached data -> bgt.geojson')
bgt = gpd.read_file(fname)

## Download precipitation and evaporation
We download precipitation and evaporation from the KNMI and calculate the net recharge in each timestep.

In [ ]:
knmi_ds = nlmod.read.knmi.get_recharge(ds, cachedir=cachedir, cachename="recharge.nc")
ds.update(knmi_ds)

## Create a Modflow 6 simulation
We generate a Modflow 6 simulation (sim) abd groundwater flow (gwf) model.

In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim, complexity='moderate')

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
dis = nlmod.gwf.dis(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=0.0)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

# create storage package
sto = nlmod.gwf.sto(ds, gwf)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

## Process surface water
We grid the surface water elements, calculate the conductane, and split the river Lek from the other shapes.

In [ ]:
bgt_grid = nlmod.grid.gdf_to_grid(bgt, ds).set_index("cellid")

bed_resistance = 1.0
bgt_grid["cond"] = bgt_grid.area / bed_resistance

# handle the lek as a river
mask = bgt_grid["bronhouder"] == "L0002"
lek = bgt_grid[mask]
bgt_grid = bgt_grid[~mask]

## Add a RIV-package
We add the river Lek using the River-package, which means it can infiltrate as well.

In [ ]:
lek["stage"] = 0.0
lek["rbot"] = -3.0
spd = nlmod.gwf.surface_water.build_spd(lek, "RIV", ds)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data={0: spd})

## Add a DRN-package
We add all other surface water elements as drains, which means they cannot infiltrate.

In [ ]:
drn = nlmod.gwf.surface_water.gdf_to_seasonal_pkg(bgt_grid, gwf, ds, save_flows=True);

## Run and process output

In [ ]:
nlmod.sim.write_and_run(sim, ds)

Retreive output from the model

In [ ]:
head0 = nlmod.gwf.get_heads_da(ds)
# Get the groundwater level, which is the head in the upper active layer (not all layers are present everywhere)
gws0 = nlmod.gwf.output.get_gwl_from_wet_cells(head0)
q_drn0 = nlmod.gwf.output.get_budget_da('DRN', ds=ds)

## Plot the groundwater level

In [ ]:
f, ax = nlmod.plot.get_map(extent)
pc = nlmod.plot.data_array(gws0.mean("time"), ds=ds)
cbar = nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none");

Let's plot the heads at the office of Artesia, in the centre of Schoonhoven.

In [ ]:
x = 118228
y = 439870
head_point = nlmod.gwf.get_head_at_point(head0, x=x, y=y, ds=ds).to_pandas()
ax = head_point.plot(figsize=(13,8))
ax.figure.tight_layout(pad=0.0)

Plot the discharge from the model

In [ ]:
f, ax = nlmod.plot.get_map(extent)
da = -q_drn0.sum('layer').mean('time') / ds['area']
pc = nlmod.plot.data_array(da, ds=ds, cmap='Blues')
cbar = nlmod.plot.colorbar_inside(pc, label='Drainagedebiet (mm/d)')
bgt.plot(ax=ax, edgecolor="k", facecolor="none");

## Now calculate with a higher stage of the Lek
We now want to know what is the influence of the river Lek. Therefore we set the stage of the Lek 1 meter higher, from 0.0 to 1.0 m NAP.

In [ ]:
# remove the current RIV-package
gwf.remove_package('RIV')
# set the stage of the Lek
lek["stage"] = 1.0
# generate a new riv package
spd = nlmod.gwf.surface_water.build_spd(lek, "RIV", ds)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data={0: spd})
# run the model again
nlmod.sim.write_and_run(sim, ds)

Retreive output from the model

In [ ]:
head1 = nlmod.gwf.get_heads_da(ds)
gws1 = nlmod.gwf.output.get_gwl_from_wet_cells(head1)
q_drn1 = nlmod.gwf.output.get_budget_da('DRN', ds=ds)

## Plot the difference in groundwater level between the two model runs

In [ ]:
f, ax = nlmod.plot.get_map(extent)
dgws = gws1 - gws0
pc = nlmod.plot.data_array(dgws.mean("time"), ds=ds, vmin=0.0, vmax=1.0)
cbar = nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none");

In [ ]:
dhead = (head1 - head0).mean('time')
fg = dhead.plot(
    x="x",
    y="y",
    col="layer",
    col_wrap=5,
    subplot_kws={"aspect": "equal"},
    vmin=0.0,
    vmax=1.0
)

So the effect in the polder are minimal. Maybe the drainage near the river increases?

In [ ]:
Exercise 2: plot the increase in drain discharge as a percentage of the original drain discharge.

## Exercises

Exercise 3: do you think the model is large enough to awnser this question?

Exercise 4: the top layer of the model is the Holocene layer (HLc), for which REGIS does not contain hydraulic conductivities. Therefore kh is set to the default value of 1.0 m/d and kv to 0.1 m/d (see the info message after `nlmod.to_model_ds()`). We can use GeoTOP to get a better estimate for these conductivities. Change an argument in `nlmod.read.regis.get_combined_layer_models()` and recalculate the effect of an increase of the stage of the Lek with 1 meter. As the GeoTOP-data has more variation, with high and low hydrulic conductivities close to eachother, the model has difficulty in converging. Change the `complexity` argument in `nlmod.sim.ims()`, so the model will converge again.